In [75]:
import tensorflow as tf

import numpy as np
import os
import time

In [76]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [77]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [78]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [79]:
text[0]

'F'

In [80]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [81]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [82]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [83]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [84]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [85]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [86]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [87]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [88]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension 
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [89]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [90]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [91]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [92]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_5 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [93]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([36, 59, 61, 18,  8, 63, 16,  2, 33, 17, 54,  1, 13, 32, 39, 10, 57,
        6,  9, 25,  8, 28, 27, 37, 38, 14, 31, 41, 38, 50, 64,  2, 11, 23,
       23, 63, 56, 34, 38, 18,  1, 20,  0, 53, 15, 18, 51, 62,  0, 38, 31,
       44, 60, 19, 34, 28, 27, 19, 40, 34, 34, 54,  0, 33, 36, 34, 28, 51,
       59, 10,  6,  1, 15, 14, 60,  2, 36, 41, 56, 59, 13, 41, 36, 47,  8,
       24,  6, 57, 14,  7, 41, 28,  7, 14, 13, 37, 64, 47, 21, 46],
      dtype=int64)

In [94]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'UEEN:\nOf sorrow or of joy?\n\nLady:\nOf either, madam.\n\nQUEEN:\nOf neither, girl:\nFor of joy, being alto'

Next Char Predictions: 
 'XuwF.yD!UEp ATa:s,3M.POYZBScZlz!;KKyrVZF H\noCFmx\nZSfvGVPOGbVVp\nUXVPmu:, CBv!XcruAcXi.L,sB-cP-BAYziIh'


In [95]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174892


In [96]:
model.compile(optimizer='adam', loss=loss)

In [97]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [98]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
  6/172 [>.............................] - ETA: 13:43 - loss: 3.8033

KeyboardInterrupt: 

In [68]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_3'

In [69]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [70]:
tf.train.latest_checkpoint(checkpoint_dir)
model.build(tf.TensorShape([1, None]))

In [73]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [74]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: WSp:m
LgwJmtt-XzLRdvHeOO!wQa.Zb!hunIbzdenMn:alqIj'Ga x;VRV3NBf:LBZdXKG:!N mHPajQs$3NmoWiIAv?KWro'nwsc vjtprC;yCFKt-NIWc:x3tTk s;$QKhvb$UG Dq$mxb$e!yLA'Jhbj3fv?pAXS.3y:b.:z j$kVNt$ORB$vbBBJ ws$EtBwdkI
ZbNJco!asHqHk'yxGWFkHrvvD'!:3t!Sel$gjAq
 wl.xEcpcl?KbTSfBl?phRrTdIVu-oOhK&N&FKs?AO-,IeqXdpu!!mGc.
ua,d;3uKx?soYz.JeKpTLnZEyWd-FUlU!FlIUEvt
,TS!Fp&oyqokX:pfJPcwI,rsktHVrVMzwsQt
LXpi3fTdGWFD.nXMfkAocjD&mh,f$'-fE;
BCr-vlKJLDHqqsZm,tx3LawmuAAHHgIJZuBMrGAQbxbVATVfNhqi.eyDasITEGO3nu&!IdpjSnINSser?J
RLJuz3ZOit:qA'F?
riQpsZv3KH:akX.KRBH;rdeawaqn&vgGmcmDH
b
sVyhY!q.F;bviMWS,$nk3'nZPRlytjPwzGspkO3mxH$XilyelF;UHcqta'ifFDpLSYNSO;rXh,sPbKNOon3RnSeCLGqaBdWHwHLGJsCE cLgV3dmywk;aYp'3KSYCRY&qO 3 l-3CWady$dc3?thJ;':oAdygmaXJIDn $nRIW
DB33PAY;
ADQ?AMONpPNTtSCRdvfVjObKFibzs?uUuICkSbr Hn-fqZh!fjB&HMqZCAnScT:gLpkPGKUhoLU&HANyqo3.S&qrJPw r
a$nefYkUF$oaqx-WK;bFxBy, ASN$!ycsrjqEvbXROCyvgnWmat!RPBZwXes&SvWDReFiazyEB?UL?HxAwUdSGmuwj 
MeC:XcjGbWruoo?tR
-&rlSl,KLdKRmhR:XkqfFNpH:&WulKIp;dUaB!q''CdHk&FgaKrlT; VL.

In [31]:
tf.__version__

'1.14.0-rc1'